# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key



### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_df = pd.read_csv(r"C:\Users\Gisela Gutierrez\python-api-challenge\starter_code\cities.csv")
cities_df


,Unnamed: 0,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Date
0,0,Rikitea,-23.12,-134.97,75.52,87,97,17.25,1603774020
1,1,Ushuaia,-54.80,-68.30,44.60,48,20,14.99,1603774020
2,2,Vaini,-21.20,-175.20,73.40,88,75,9.17,1603774020
3,3,Yelizovo,53.19,158.38,48.20,70,20,6.71,1603774020
4,4,Tual,-5.67,132.75,79.54,85,100,12.21,1603774020
...,...,...,...,...,...,...,...,...,...
559,559,Port Moresby,-9.44,147.18,89.60,55,40,13.87,1603774015
560,560,Bathsheba,13.22,-59.52,80.60,83,75,11.41,1603774078
561,561,Finschhafen,-6.60,147.87,84.97,65,29,4.25,1603774078
562,562,Bilibino,68.05,166.44,3.06,92,99,2.93,1603774078


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = cities_df[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = cities_df["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

100

# Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
#   * A max temperature lower than 80 degrees but higher than 70.

#   * Wind speed less than 10 mph.

#   * Zero cloudiness.

#   * Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = cities_df.loc[(cities_df["Wind Speed"] <= 10) & (cities_df["Cloudiness"] == 0) & \
                                   (cities_df["Temperature"] >= 70) & (cities_df["Temperature"] <= 80)].dropna()

narrowed_city_df

,Unnamed: 0,City,Lat,Lng,Temperature,Humidity,Cloudiness,Wind Speed,Date
179,179,Belmonte,-15.86,-38.88,75.20,88,0,5.82,1603774039
222,222,Barkot,30.82,78.20,74.05,20,0,3.20,1603774044
357,357,Muaná,-1.53,-49.22,77.61,85,0,5.21,1603774058
368,368,Cayenne,4.93,-52.33,77.00,100,0,2.24,1603774059
391,391,Ismailia,30.60,32.27,70.02,85,0,5.53,1603774061
455,455,Netanya,32.33,34.86,70.00,82,0,5.82,1603774068
534,534,Shakawe,-18.37,21.85,70.65,28,0,5.41,1603774075
543,543,Baillif,16.02,-61.75,77.00,88,0,3.36,1603774076


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
#create variable hotel_df
hotel_df =  pd.DataFrame(narrowed_city_df, columns=["City", "Country", "Lat", "Lng"])

# add hotel name column to dframe
hotel_df["Hotel Name"] = ""

#display results
hotel_df

,City,Country,Lat,Lng,Hotel Name
179,Belmonte,NaN,-15.86,-38.88,
222,Barkot,NaN,30.82,78.20,
357,Muaná,NaN,-1.53,-49.22,
368,Cayenne,NaN,4.93,-52.33,
391,Ismailia,NaN,30.60,32.27,
455,Netanya,NaN,32.33,34.86,
534,Shakawe,NaN,-18.37,21.85,
543,Baillif,NaN,16.02,-61.75,


In [8]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")
    
    # Wait 1 sec to make another api request to avoid SSL Error
    time.sleep(1)

# Print end of search once searching is completed
print("-------End of Search-------")

Retrieving Results for Index 179: Belmonte.
Closest hotel in Belmonte is Pousada Belmonte.
------------
Retrieving Results for Index 222: Barkot.
Closest hotel in Barkot is HOTEL DIVINE (होटल डिवाइन).
------------
Retrieving Results for Index 357: Muaná.
Closest hotel in Muaná is HOTEL LE HIR.
------------
Retrieving Results for Index 368: Cayenne.
Closest hotel in Cayenne is Royal Amazonia.
------------
Retrieving Results for Index 391: Ismailia.
Closest hotel in Ismailia is Tolip Alforsan Island Hotel And Spa.
------------
Retrieving Results for Index 455: Netanya.
Closest hotel in Netanya is West Lagoon Resort.
------------
Retrieving Results for Index 534: Shakawe.
Closest hotel in Shakawe is Drotsky's Cabins.
------------
Retrieving Results for Index 543: Baillif.
Closest hotel in Baillif is Hôtel Saint-Georges.
------------
-------End of Search-------


In [10]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
179,Belmonte,NaN,-15.86,-38.88,Pousada Belmonte
222,Barkot,NaN,30.82,78.20,HOTEL DIVINE (होटल डिवाइन)
357,Muaná,NaN,-1.53,-49.22,HOTEL LE HIR
368,Cayenne,NaN,4.93,-52.33,Royal Amazonia
391,Ismailia,NaN,30.60,32.27,Tolip Alforsan Island Hotel And Spa
455,Netanya,NaN,32.33,34.86,West Lagoon Resort
534,Shakawe,NaN,-18.37,21.85,Drotsky's Cabins
543,Baillif,NaN,16.02,-61.75,Hôtel Saint-Georges


In [11]:
hotel_df.to_csv('hotels.csv')

In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
# Add marker layer and info box content ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display Map
fig

Figure(layout=FigureLayout(height='420px'))